# Retention API for TEOTIL3

Outline code for API providing retention/transmission estimates from any location in Norway to the coast. See e-mail from Torstein Finnesand received 21.11.2023 at 07.34 for details.

Needs refining and deploying. Also consider making it standards-compliant using [pygeoapi](https://pygeoapi.io/), as suggested by Kim.

In [1]:
import geopandas as gpd
from fastapi import FastAPI
from fastapi.testclient import TestClient
from pydantic import BaseModel
from shapely.geometry import Point

GPKG_PATH = r"/home/jovyan/shared/common/teotil3/core_data/tidied/teotil3_data.gpkg"

app = FastAPI(docs_url="/", title="TEOTIL3 API")


class Coordinate(BaseModel):
    easting: float
    northing: float


@app.post("/transmission_to_coast")
async def transmission_to_coast(coordinate: Coordinate):
    # Create point
    point = Point(coordinate.easting, coordinate.northing)

    # Get polygons
    gdf = gpd.read_file(
        GPKG_PATH,
        layer="regines",
        driver="GPKG",
        bbox=[point.x, point.y, point.x, point.y],
    )

    # Find polygon containing point
    for index, row in gdf.iterrows():
        if row["geometry"].contains(point):
            data = row.to_dict()
            del data["geometry"]
            return data

    return {"error": "No polygon contains the provided point"}


# Testing
client = TestClient(app)

# Langtjern co-ords
east, north = 209438.18851, 6704338.36601
coords = {"easting": east, "northing": north}

# Query API
response = client.post("/transmission_to_coast", json=coords)
print(response.json())

{'regine': '012.CB5D', 'vassom': '012', 'a_cat_poly_km2': 4.66759, 'upstr_a_km2': 4.67, 'ospar_region': 'Skagerrak', 'komnr_2014': '0615', 'fylnr_2014': '06', 'komnr_2015': '0615', 'fylnr_2015': '06', 'komnr_2016': '0615', 'fylnr_2016': '06', 'komnr_2017': '0615', 'fylnr_2017': '06', 'komnr_2018': '0615', 'fylnr_2018': '06', 'komnr_2019': '0615', 'fylnr_2019': '06', 'komnr_2020': '3039', 'fylnr_2020': '30', 'komnr_2021': '3039', 'fylnr_2021': '30', 'komnr_2022': '3039', 'fylnr_2022': '30', 'komnr_2023': '3039', 'fylnr_2023': '30', 'a_agri_km2': 0.0, 'a_glacier_km2': 0.0, 'a_lake_km2': 0.26116, 'a_other_km2': 0.0, 'a_sea_km2': 0.0, 'a_upland_km2': 0.53124, 'a_urban_km2': 0.0, 'a_wood_km2': 3.87519, 'ar50_tot_a_km2': 4.66759, 'a_cat_land_km2': 4.66759, 'a_lake_nve_km2': 0.311034, 'runoff_mm/yr': 569, 'q_cat_m3/s': 0.08416}
